# Hyper Island Data Analysis Workshop


### Setup Linux

On any distro that includes Python you should be able to just create a directory, `cd` into it and run the following commands:

```
python3 -m venv env
source env/bin/activate
python3 -m pip install -r requirements.txt
```

You can then run `jupyter notebook` to start the notebook server and access it from the browser using the URL that will be displayed in the terminal.

The `matplotlib` library might cause some issues, I had to install it system-wide using:

```
sudo apt install python3-matplotlib
```

### Setup Windows

On Windows you need to make sure to install Python first. Get the latest version of Python 3 from https://www.python.org/downloads/windows/.
This release will include the package manager `pip`.

You should then create a folder and download the workshop files into it. The process of installing all dependencies should be done from the command line, using for example `cmd` or `PowerShell`. I opted for `cmd`, using the following commands to create and activate a virtual environment:

```
py -m venv env
cd env/Scripts
activate.bat
cd ../..
```

At the beginning of your command prompt you should now read `(env)`, indicating that you activated this environment. You can now install the dependencies (provided that you downloaded the `requirements.txt` file into the folder you are using) with the following commands:

```
py -m pip install -r requirements
py -m pip install notebook
```

`notebook` appears to be Windows-specific, so I'm not including it in the `requirements.txt` file.

Once the installation finishes, you can start the notebook server using:

```
py -m notebook
```

Then go to the browser (it might open by itself, otherwise use the URL from the cmd-prompt) and create a new notebook for your analysis.

### Getting Started: Imports I'll be Using

The `requirements.txt` file gives you an idea of which libraries I'll be using for my work. You can import them as follows:

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import requests
import datetime

from pandas_datareader import data as pdr
from bs4 import BeautifulSoup
from scipy import stats

### Data Source: Direct Download

Instead of scraping the data by hand we are using a direct download from https://www.folkhalsomyndigheten.se/smittskydd-beredskap/utbrott/aktuella-utbrott/covid-19/statistik-och-analyser/bekraftade-fall-i-sverige/

## Data Cleaning and Preparation

First step: Preview the data to manually identify any possible issues and get and idea of what the data look like.

![Data preview](data.png)

### Loading and Combining Everything

Assuming that possible errors or missing data are going to be somewhat consistent across the data, so not point in fixing that individually per file.

In [ ]:
# standard utf-8 encoding does not work for Swedish
# https://docs.python.org/3/library/codecs.html#standard-encodings
c1 = pd.read_csv("data/Covid_Statistics_W6_10.csv", encoding="Latin-1")

# 'display()' command is a part of IPython / Jupyter
# use to achieve multiple outputs from one cell
# don't use 'print()' inside display because of its None-return value
display(f"Number of rows: {len(c1.index)}") # supports f-strings
display(c1.head())

In [ ]:
# reading data into a dictionary of the form {'cN': DataFrame}
data = dict()
data['c1'] = c1

data['c2'] = pd.read_csv("data/Covid_Statistics_W11_20.csv", encoding="Latin-1")
data['c3'] = pd.read_csv("data/Covid_Statistics_W21_30.csv", encoding="Latin-1")
data['c4'] = pd.read_csv("data/Covid_Statistics_W31_39.csv", encoding="Latin-1")
data['c5'] = pd.read_csv("data/Covid_Statistics_W40_48.csv", encoding="utf-8")

# concat takes an iterable and in case of dictionary orders by key
df = pd.concat(data)

display(f"Number of rows: {len(df.index)}")
display(df.head())

# looking at types to see where there might be data quality issues
display(df.dtypes)

In [ ]:
df.tail()

### Cleaning Numeric Columns Imported as Objects

In [ ]:
# finding which rows in a particular column are not numeric strings (only contain numbers)
display(df['tot_antal_fall'][~df['tot_antal_fall'].str.isnumeric()])
display(df['nya_fall_vecka'][df['nya_fall_vecka'].str.isnumeric() == False])

In [ ]:
# how to treat those values? let's take a look at summary statistics to get an idea
# creating a new dataframe containing only the numeric data from those columns
tot_antal_num = df['tot_antal_fall'][df['tot_antal_fall'].str.isnumeric()].apply(pd.to_numeric)
nya_fall_num = df['nya_fall_vecka'][df['nya_fall_vecka'].str.isnumeric() == True].apply(pd.to_numeric)
num_data = pd.DataFrame.from_dict({'tot_antal_fall': tot_antal_num, 'nya_fall_vecka': nya_fall_num})

# using the 'aggregate' function to create specific summary statistics (subset of 'describe')
num_data.agg({"tot_antal_fall": ["min", "max", "median"], "nya_fall_vecka": ["min", "max", "median"]})

In [ ]:

# handle NaN values first -> fill with 0 for visualization later
df = df.fillna(0)

# for 'total_antal_fall' let's use a random integer between 0 and 15
df['tot_antal_fall'][~df['tot_antal_fall'].str.isnumeric()] = df['tot_antal_fall'][~df['tot_antal_fall'].str.isnumeric()].apply(lambda x: np.random.randint(0, 15))

# something seems off about 'nya_fall_vecka', so let's just replace them all with 0
df['nya_fall_vecka'][df['nya_fall_vecka'].str.isnumeric() == False] = 0

In [ ]:
# making the columns in the original dataframe numeric for further processing
df['tot_antal_fall'] = df['tot_antal_fall'].astype(int)

# first need to remove NaN values
df['nya_fall_vecka'] = pd.to_numeric(df['nya_fall_vecka'], errors='coerce')
df['nya_fall_vecka'] = df['nya_fall_vecka'].astype(int)

display(df.dtypes)


### Handling Extreme Values

In [ ]:
# there's extreme values in the nya_fall_vecka column
df.plot.scatter(x='veckonummer', y='nya_fall_vecka')

How do we address this issue? One approach is to exclude values that fall outside 2 or 3 standard deviations:

![z-score](The_Normal_Distribution.svg)

In [ ]:
# let's remove extreme values in the 'nya_fall_vecka' column based on z-score
# https://www.kite.com/python/answers/how-to-remove-outliers-from-a-pandas-dataframe-in-python
df = df[(np.abs(stats.zscore(df.nya_fall_vecka)) < 3)]

In [ ]:
# let's check if there are still extreme values present
df.plot.scatter(x='veckonummer', y='nya_fall_vecka')

In [ ]:
# let's apply the same function again to remove the distinctly different value
df = df[(np.abs(stats.zscore(df.nya_fall_vecka)) < 3)]

In [ ]:
df.plot.scatter(x='veckonummer', y='nya_fall_vecka')

In [ ]:
# compare to 10948 rows in the beginning
display(len(df))

### Spellchecking the 'Kommun' Column

In [ ]:
# planning on grouping by 'kommune' since 'stadsdel' is too granular
# need to check whether all the 'Kommun' are actually correct. So let's remove the statsdel and check the rest against a valid source
df['Kommun'] = df.Kommun_stadsdel.apply(lambda x: x.split()[0])

# Norwegian Wikipedia article on Swedish municipalities contains a table, so using that as a source
source = 'https://no.wikipedia.org/wiki/Sveriges_kommuner#Kommunene:_M'

# time to break out those web-scraping skills
# don't forget to pip-install 'beautifulsoup4'!

website = requests.get(source).text
soup = BeautifulSoup(website,'lxml')

tbl = soup.find('table')
print(tbl)

In [ ]:
# but then again... pandas allows us to do that directly :)
source_kommuner = pd.read_html(source)[0]

# so let's use that and look for the values that are in our DF but not in 'source_kommuner'
offenders = df.Kommun[df.Kommun.isin(source_kommuner.Kommune) == False]
print(offenders.unique())

In [ ]:
replacement = dict.fromkeys(offenders, None)
replacement['Falun'] = 'Falu'
replacement['Gbg'] = 'Göteborg'
replacement['Lilla'] = 'Lilla Edet' # this mistake is due to assuming kommune-names are always one word
replacement['Malmös'] = 'Malmö'
# replacement['Malung'] = 'Malung-Sälen' # this mistake is due to assuming kommune-names are always one word
replacement['Skhlm'] = 'Stockholm'
replacement['Stockholms'] = 'Stockholm'
replacement['Upplands'] = 'Upplands Väsby' # this mistake is due to assuming kommune-names are always one word
replacement['Östra'] = 'Östra Göinge' # this mistake is due to assuming kommune-names are always one word

# could we find better way of extracting 'kommune' name?

# replace wrong names with corrections in DF
df['Kommun'] = df['Kommun'].map(replacement).fillna(df['Kommun'])

# try to find 'offenders' again to see if it worked
display(df.Kommun[df.Kommun.isin(source_kommuner.Kommune) == False])

In [ ]:
# export for use in Tableau
df.to_csv("export.csv")

## Data Analysis

Main analysis will be done visually in Tableau, but let's take a quick look to verify that the data makes sense.

In [ ]:
# let's take a look at overall cases and new cases per week in two major cities
df[df['Kommun'].isin(['Stockholm'])].groupby(['veckonummer', 'Kommun']).agg(
    sum_new_cases=pd.NamedAgg(column="nya_fall_vecka", aggfunc=sum),
    sum_overall_cases=pd.NamedAgg(column="tot_antal_fall", aggfunc=sum)
).plot.line(subplots=True)

df[df['Kommun'].isin(['Göteborg'])].groupby(['veckonummer', 'Kommun']).agg(
    sum_new_cases=pd.NamedAgg(column="nya_fall_vecka", aggfunc=sum),
    sum_overall_cases=pd.NamedAgg(column="tot_antal_fall", aggfunc=sum)
).plot.line(subplots=True)

### Step-by-Step Pivot Table

In [ ]:
# Pivot table can be more intuitive to work with
# questions we might want to answer:
# * are case numbers per inhabitant similar across municipalities?
# * how have cases developed over time?
# * ...

# good step-by-step tutorial at https://pbpython.com/pandas-pivot-table-explained.html

# simplest pivot is chosing an index; here taking two columns as index to group by municipality and week
pd.pivot_table(df, index=["Kommun", "veckonummer"])

In [ ]:
# selecting the columns that show numbers per inhabitant
pd.pivot_table(df, index=["Kommun", "veckonummer"], values=["antal_fall_per10000_inv", "tot_antal_fall_per10000inv"], aggfunc=np.sum)

In [ ]:
# in order to do some more interesting work with the pivot table, let's add the country for each row
# getting country information from https://en.wikipedia.org/wiki/List_of_municipalities_of_Sweden#List:_M

# create new DF with county info
source_counties = pd.read_html("https://en.wikipedia.org/wiki/List_of_municipalities_of_Sweden#List:_M")[1]
counties = source_counties[["Municipality", "County"]]
counties['Municipality'] = counties['Municipality'].str.replace(" Municipality", "")
counties['County'] = counties['County'].str.replace(" County", "")

# combine DFs so that county is included alongside municipality
combined = pd.merge(df, counties, left_on="Kommun", right_on="Municipality", how="inner").drop("Kommun", axis=1)

In [ ]:
pd.set_option("max_rows", None)

# pivot table for that combination
pd.pivot_table(combined, index=["County", "Municipality"], values=["antal_fall_per10000_inv", "tot_antal_fall_per10000inv"], aggfunc=[np.sum, np.mean])

In [ ]:
pd.set_option("max_rows", 20)

### Adding Finance Information

Let's quickly add an additional data source to compare the Covid-data against by using Yahoo Finance.
https://github.com/ranaroussi/yfinance

In [ ]:
# OMX -> 30 most traded stocks at Stockholm stock exchange
omx = yf.Ticker("^OMX")
omx_history = omx.history(period="11mo")
omx_history

In [ ]:
# we want to use this in with pandas, so let's hijack the pandas datareader to use yfinance
# this will make pdr output the data in a format we can use, but be faster because it uses yfinance
yf.pdr_override()

omx_data = pdr.get_data_yahoo("^OMX", start="2020-01-01", end="2020-12-07")
display(omx_data.head())
display(omx_data.Close.plot())

In [ ]:
df[df['Kommun'].isin(['Stockholm'])].groupby(['veckonummer', 'Kommun']).agg(
    sum_new_cases=pd.NamedAgg(column="nya_fall_vecka", aggfunc=sum)
).plot.line()

In [ ]:
stockholm = pd.DataFrame(df[df['Kommun'].isin(['Stockholm'])].groupby(['veckonummer', 'Kommun']).agg(
    sum_new_cases=pd.NamedAgg(column="nya_fall_vecka", aggfunc=sum),
    veckonummer=pd.NamedAgg(column="veckonummer", aggfunc=min)
))

In [ ]:
stockholm['Date'] = stockholm.apply(lambda row: datetime.datetime.strptime(f"2020-W{row.veckonummer}-1", "%Y-W%W-%w"), axis=1)
stockholm = stockholm.drop("veckonummer", axis=1)

In [ ]:
stockholm = stockholm.set_index("Date")
omx_data = omx_data.reset_index()
omx_data = omx_data.set_index("Date")

In [ ]:
stockholm.corrwith(omx_data.Close)